## EyeON: Eye on Operational Networks
### a firmware inventory/threat analysis tool


In [ ]:
from eyeon import observe
from pprint import pprint

### Objects
EyeON consists of two parts - an `observe` call and a `parse` call. `observe` works on a single file to return a suite of identifying metrics. `parse` calls `observe` recursively, returning an observation for each file in a directory. Both of these can be run either from a library import or a `CLI` command.

In [ ]:
obs = observe.Observe("./tests/Obsidian.1.1.9.exe")
ols = observe.Observe("./tests/ls")

### Data Standard
Depending on the file type, e.g. PE or ELF, different observations will be collected. 
For instance, PE files typically contain more metadata and have signature information.

In [ ]:
for key, val in obs.signatures["signatures"].items():
    for cert in val["certs"]:
        for dat in cert.split("\n"):
            print(dat)

In [ ]:
import lief
obsi = lief.parse("./tests/Obsidian.1.1.9.exe")


In [ ]:
if obsi.has_resources:
    rsrc_dir = obsi.data_directory(lief.PE.DATA_DIRECTORY.RESOURCE_TABLE)
    if rsrc_dir.has_section:
        print(rsrc_dir.section)

In [ ]:
for i in obsi.resources.childs:
    print(i.id)
    print(i)

In [ ]:
idn = i.childs[0]
print(idn)
lgn = idn.childs[0]
print(lgn)
manifest = bytes(lgn.content).decode("utf-8")
print(manifest)

In [ ]:
from xml.etree import cElementTree as ET
e = ET.XML(manifest)
pprint(etree_to_dict(e))

In [ ]:
from surfactant.infoextractors.pe_file import extract_pe_info
filedata = extract_pe_info("./tests/Obsidian.1.1.9.exe")
filedata

In [ ]:

def etree_to_dict(t):
    d = {t.tag: {} if t.attrib else None}
    children = list(t)
    if children:
        dd = defaultdict(list)
        for dc in map(etree_to_dict, children):
            for k, v in dc.items():
                dd[k].append(v)
        d = {t.tag: {k:v[0] if len(v) == 1 else v for k, v in dd.items()}}
    if t.attrib:
        d[t.tag].update(('@' + k, v) for k, v in t.attrib.items())
    if t.text:
        text = t.text.strip()
        if children or t.attrib:
            if text:
              d[t.tag]['#text'] = text
        else:
            d[t.tag] = text
    return d

In [ ]:
# print(obsi.resources.childs[1])
# # print(get_content(obsi.resources.childs[1]))
# print(obsi.resources.childs[1].childs[0])
# # print(get_content(obsi.resources.childs[1].childs[0]))
# print(obsi.resources.childs[1].childs[0].childs[0])
# print(get_content(obsi.resources.childs[1].childs[0].childs[0]))
print(obsi.resources_manager.manifest)

In [ ]:
G = nx.DiGraph(name="Signing Certs")
label_dict = {}
sample_cert = None


In [ ]:
#for root, dirs, files in os.walk(exe):
# for file in os.listdir(exe):
#    path = root.split(os.sep)
#    print((len(path) - 1) * '---', os.path.basename(root))
#   for file in files:
#    print(len(path) * '---', file)
file = "Obsidian.1.1.9.exe"
pe = lief.parse(f"tests/{file}")
# for sig in pe.signatures:
sig = pe.signatures[0]
for crt in sig.certificates:
# print(crt)
# Write cert (in DER format)
    with open(f"tests/outputs/{crt.subject}.crt", "wb") as binary_file:
      binary_file.write(crt.raw)
    # Add to graph
    try:
      G.add_nodes_from([crt.subject,crt.issuer])
      G.add_edge(crt.subject,crt.issuer)
      label_dict[crt.subject]=crt.subject
      G.add_nodes_from([crt.subject,crt.issuer])
      G.add_edge(crt.subject,crt.issuer)
    except nx.NetworkXError:
      print('Yikes!')
      print(crt.subject)
      print(crt.issuer)
    spl = str(crt.subject).split(', ')
    subj = {}
    for i in spl:
        j = i.split("=")
        try:
            subj[j[0]] = j[1]
        except:
            print(j)
            subj[j[0]] = ""
    print(crt.subject)
    print(subj)


In [ ]:
nx.draw_spring(G,with_labels=True, labels=label_dict, )


In [ ]:
# %pip install matplotlib

In [ ]:
G.in_degree

In [ ]:
spl = str(crt.subject).split(',')
subj = {}
for i in spl:
    j = i.split("=")
    subj[j[0]] = j[1]


In [ ]:
subj

In [ ]:
help(nx.draw_networkx)